# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

burdan sonraki kısım ucak rezervasyonu için ekledim

In [ ]:
# 1. Gerekli kütüphaneleri yükle
import os
import json
importuçuş rezervasyon random
import string
from datetime import datetime
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr


In [ ]:
# 2. Ortam değişkenlerini yükle
load_dotenv(override=True)
openai = OpenAI()
MODEL = "gpt-4o-mini"


In [ ]:
# 3. Uçak bileti fiyatları sözlüğü (çift şehirli anahtarlarla)
ticket_prices = {
    ("istanbul", "berlin"): "$499",
    ("istanbul", "tokyo"): "$1400",
    ("ankara", "berlin"): "$550",
    ("izmir", "tokyo"): "$1350",
    ("antalya", "paris"): "$790",
    ("istanbul", "paris"): "$899"
}


In [ ]:

# 4. Bilet fiyatı fonksiyonu (güvenli)
def get_ticket_price(departure_city, destination_city):
    if not departure_city or not destination_city:
        return "Error: both departure and destination cities are required."
    key = (departure_city.lower(), destination_city.lower())
    price = ticket_prices.get(key)
    return price if price else f"Estimated price from {departure_city.title()} to {destination_city.title()} is around $1000."

In [ ]:

# 5. PNR üretici & rezervasyon fonksiyonu (tarih destekli)
def generate_pnr():
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=6))

def make_reservation(departure_city, destination_city, travel_date):
    pnr = generate_pnr()
    # travel_date format 'YYYY-MM-DD'
    try:
        fly_date = datetime.fromisoformat(travel_date)
        date_str = fly_date.strftime("%d %B %Y")
    except:
        date_str = travel_date
    return (
        f"Your reservation from {departure_city.title()} to {destination_city.title()} on {date_str} is confirmed.\n"
        f"Flight departs at 13:45 local time.\n"
        f"Reservation Code (PNR): {pnr}.\n"
        f"Seat selection will be available during check-in."
    )

In [ ]:
# 6. Fonksiyon sarmalayıcıları
def get_ticket_price_fn(departure_city: str, destination_city: str) -> str:
    return get_ticket_price(departure_city, destination_city)

def make_reservation_fn(departure_city: str, destination_city: str, travel_date: str) -> str:
    return make_reservation(departure_city, destination_city, travel_date)


In [ ]:

# 7. Fonksiyon tanımları
functions = [
    {
        "name": "get_ticket_price",
        "description": "Returns the ticket price between a departure and destination city.",
        "parameters": {
            "type": "object",
            "properties": {
                "departure_city": {"type": "string", "description": "Departure city."},
                "destination_city": {"type": "string", "description": "Destination city."}
            },
            "required": ["departure_city", "destination_city"]
        }
    },
    {
        "name": "make_reservation",
        "description": "Reserves a flight from departure to destination city on a given date and returns reservation details.",
        "parameters": {
            "type": "object",
            "properties": {
                "departure_city": {"type": "string", "description": "Departure city."},
                "destination_city": {"type": "string", "description": "Destination city."},
                "travel_date": {"type": "string", "description": "Travel date in YYYY-MM-DD format."}
            },
            "required": ["departure_city", "destination_city", "travel_date"]
        }
    }
]

In [ ]:
# 8. Sistem mesajı
system_message = (
    "You are a helpful assistant for an airline called FlightAI. "
    "Never call any functions unless both the departure city, destination city, and travel date are clearly provided. "
    "If the user mentions only destination or departure, ask for the missing details. "
    "Ask explicitly for travel date in YYYY-MM-DD format."
)


In [ ]:

# 9. Ana sohbet fonksiyonu
def chat(message, history):
    msgs = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    # Modeli çağır ve varsa fonksiyon isteği al
    response = openai.chat.completions.create(
        model=MODEL,
        messages=msgs,
        functions=functions
    )
    msg = response.choices[0].message
    # Eğer fonksiyon çağırıldıysa
    if msg.function_call:
        fname = msg.function_call.name
        args = json.loads(msg.function_call.arguments)
        # Parametre kontrolü
        if fname == "get_ticket_price":
            result = get_ticket_price_fn(**args)
        else:
            # Önce rezervasyon için tarih parametresini al
            result = make_reservation_fn(**args)
        # Fonksiyon çağrı ve yanıtını ekle
        msgs.append({"role": "assistant", "content": None, "function_call": msg.function_call.to_dict()})
        msgs.append({"role": "function", "name": fname, "content": json.dumps(result)})
        # Son cevabı al
        response = openai.chat.completions.create(model=MODEL, messages=msgs)
        return response.choices[0].message.content
    return msg.content


In [ ]:
# 10. Gradio arayüzünü başlat
gr.ChatInterface(fn=chat, type="messages").launch()